In [8]:
import os

In [9]:
%pwd

'c:\\Users\\Sumeet\\OneDrive - Worcester Polytechnic Institute (wpi.edu)\\Chicken-Disease-Classification'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sumeet\\OneDrive - Worcester Polytechnic Institute (wpi.edu)\\Chicken-Disease-Classification'

In [29]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [30]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [31]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [32]:
import time
from pathlib import Path

In [33]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = Path(self.config.tensorboard_root_log_dir) / f"tb_logs_at_{timestamp}"
        return tf.keras.callbacks.TensorBoard(log_dir=str(tb_running_log_dir))
    
    @property
    def _create_ckpt_callbacks(self):
        # Ensure the file path ends with .weights.h5
        checkpoint_filepath = self.config.checkpoint_model_filepath
        checkpoint_filepath = str(checkpoint_filepath)  # Convert Path object to string
        if not checkpoint_filepath.endswith('.weights.h5'):
            checkpoint_filepath = checkpoint_filepath.rsplit('.', 1)[0] + '.weights.h5'
        
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_best_only=True,
            save_weights_only=True  # Save weights only to ensure the format is .weights.h5
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [34]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [35]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.optimizer = None
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        # Initialize a new optimizer
        self.optimizer = tf.keras.optimizers.Adam()

        # Recompile the model with the new optimizer
        self.model.compile(
            optimizer=self.optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [36]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-07-27 07:01:12,892: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-27 07:01:12,894: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-27 07:01:12,896: INFO: common: created directory at: artifacts]
[2024-07-27 07:01:12,897: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-07-27 07:01:12,898: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-07-27 07:01:12,899: INFO: common: created directory at: artifacts\training]
[2024-07-27 07:01:13,068: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.6997 - loss: 0.7027 - val_accuracy: 0.9375 - val_loss: 0.2914
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/

c:\Users\Sumeet\anaconda3\envs\chicken\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 1.0000 - loss: 0.0718 - val_accuracy: 1.0000 - val_loss: 0.0154
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.9432 - loss: 0.2105 - val_accuracy: 0.9531 - val_loss: 0.2456
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.9375 - loss: 0.1792 - val_accuracy: 1.0000 - val_loss: 0.0135
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9770 - loss: 0.0825 - val_accuracy: 0.9375 - val_loss: 0.2270
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 1.0000 - loss: 0.0693 - val_accuracy: 1.0000 - val_loss: 0.0152
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9640 - loss: 0.1202 - val_accuracy: 0.9531 - val_loss: 0.2649
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - accuracy: 0.9375 - loss: 0.1591 - val_accuracy: 1.0000 - val_loss: 0.0162
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9547 - loss: 0.1088 - val_accuracy: 0.9375 - val_loss: 0.231